In [1]:
import numpy as np
from sklearn import datasets
from math import sqrt

#### relu激活函数

In [2]:
# 实现relu函数
def relu(x):
    """
    input: x(ndarray)
    output: relu(x)(ndarray)
    """
    return np.maximum(x, 0)

x = np.array([-1, 1, 2])
relu(x)

array([0, 1, 2])

#### sigmoid函数及其偏导

In [3]:
# 实现sigmoid函数
def sigmoid(x):
    """
    input: x(ndarray)
    output: sigmoid(x)(ndarray)
    """
    return 1 / (1 + np.exp(-x))


# 计算sigmoid函数偏导
def deriv_sigmoid(x):
    """
    input: x(ndarray)
    output: sigmoid(x)(ndarray)
    """
    m, n = np.shape(x)
    out = np.mat(np.zeros((m, n)))
    for i in range(m):
        for j in range(n):
            out[i, j] = sigmoid(x[i, j]) * (1 - sigmoid(x[i, j]))
    return out

#### 神经网络

In [4]:
# bp神经网络训练方法
def bp_brain(feature, label, n_hidden, maxcycle, alpha, n_output):
    """
    计算隐含层的输入
    input: feature(mat):特征
            label(mat):标签
            n_hidden(int):隐含层的节点个数
            maxCycle(int):最大的迭代次数
            alpha(float):学习率
            n_output(int):输出层的节点个数
    output: w0(mat):输入层到隐含层之间的权重
            b0(mat):输入层到隐含层之间的偏置
            w1(mat):隐含层到输出层之间的权重
            b1(mat):隐含层到输出层之间的偏置
    """
    m, n = np.shape(feature)
    # 初始化
    w0 = np.mat(np.random.rand(n, n_hidden))
    w0 = w0 * (8.0 * sqrt(6) / sqrt(n + n_hidden)) - np.mat(np.ones((n, n_hidden))) * (
        4.0 * sqrt(6) / sqrt(n + n_hidden)
    )
    b0 = np.mat(np.random.rand(1, n_hidden))
    b0 = b0 * (8.0 * sqrt(6) / sqrt(n + n_hidden)) - np.mat(np.ones((1, n_hidden))) * (
        4.0 * sqrt(6) / sqrt(n + n_hidden)
    )
    w1 = np.mat(np.random.rand(n_hidden, n_output))
    w1 = w1 * (8.0 * sqrt(6) / sqrt(n_hidden + n_output)) - np.mat(
        np.ones((n_hidden, n_output))
    ) * (4.0 * sqrt(6) / sqrt(n_hidden + n_output))
    b1 = np.mat(np.random.rand(1, n_output))
    b1 = b1 * (8.0 * sqrt(6) / sqrt(n_hidden + n_output)) - np.mat(
        np.ones((1, n_output))
    ) * (4.0 * sqrt(6) / sqrt(n_hidden + n_output))
    # 训练
    i = 0
    while i <= maxcycle:
        # TODO
        # 前向传播
        # 计算隐含层的输入
        hidden_input = feature * w0 + np.tile(b0, (m, 1))
        # 计算隐含层的输出
        hidden_output = relu(hidden_input)
        # 计算输出层的输入
        output_in = hidden_output * w1 + np.tile(b1, (m, 1))
        # 计算输出层的输出
        output_out = relu(output_in)

        # TODO
        # 反向传播
        # 隐藏层到输出层之间的残差
        delta_output = -np.multiply(
            (label - output_out), np.multiply(output_out, (1 - output_out))
        )
        # 输入层到隐含层之间的残差
        delta_hidden = np.multiply(
            delta_output * w1.T, np.multiply(hidden_output, (1 - hidden_output))
        )
        # 更新隐含层到输出层之间的权重和偏置
        w1 = w1 - alpha * (hidden_output.T * delta_output)
        b1 = b1 - alpha * np.sum(delta_output, axis=0) / m
        # 更新输入层到隐含层之间的权重和偏置
        w0 = w0 - alpha * (feature.T * delta_hidden)
        b0 = b0 - alpha * np.sum(delta_hidden, axis=0) / m

        i += 1
    return w0, b0, w1, b1

In [5]:
# 计算隐藏层的输入函数
def hidden_in(feature, w0, b0):
    m = np.shape(feature)[0]
    hidden_in = feature * w0
    for i in range(m):
        hidden_in[i,] += b0
    return hidden_in


# 计算隐藏层的输出函数
def hidden_out(hidden_in):
    hidden_out = sigmoid(hidden_in)
    return hidden_out


# 计算输出层的输入函数
def predict_in(hidden_out, w1, b1):
    m = np.shape(hidden_out)[0]
    predict_in = hidden_out * w1
    for i in range(m):
        predict_in[i,] += b1
    return predict_in


# 计算输出层的输出函数
def predict_out(predict_in):
    predict_out = sigmoid(predict_in)
    return predict_out

#### Dropout方法

In [6]:
# 由于Dropout方法输出存在随机性，我们已经设置好随机种子，你只要完成Dropout方法的实现即可
class Dropout:
    def __init__(self, dropout_ratio=0.5):
        self.dropout_ratio = dropout_ratio
        self.mask = None

    def forward(self, x, train_flg=True):
        """
        前向传播中self，mask会随机生成和x形状相同的数组，
        并将其中小于dropout_ratio的元素设为True，其余为False
        x为一个列表
        """
        # TODO
        if train_flg:
            self.mask = np.random.rand(*x.shape) > self.dropout_ratio
            return x * self.mask
        else:
            return x * (1.0 - self.dropout_ratio)

    def backward(self, dout):
        """
        前向传播时传递了信号的神经元，
        反向传播时按原样传递信号。
        前向传播没有传递信号的神经元，
        反向传播时信号就停在那里。
        dout为一个列表。
        """
        # TODO
        return dout * self.mask

In [7]:
# 测试
x = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
print(x)
dropout = Dropout(0.5)
print(dropout.forward(x))
print(dropout.backward(x))

[0 1 2 3 4 5 6 7 8 9]
[0 0 0 0 4 0 6 0 0 0]
[0 0 0 0 4 0 6 0 0 0]


#### 鸢尾花数据集分类

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

def iris_predict(train_sample, train_label, test_sample):
    """
    实现功能：1.训练神经网络
             2.预测测试样本的标签
    输入：train_sample(ndarry):训练样本的特征
            train_label(ndarry):训练样本的标签
            test_sample(ndarry):测试样本的特征
    return: test_predict(ndarry):测试样本的预测标签
    """
    # 使用交叉验证寻找最佳参数
    parameters = {
        "solver": ["lbfgs"],
        "max_iter": [500, 1000, 1500],
        "alpha": 10.0 ** -np.arange(1, 7),
        "hidden_layer_sizes": np.arange(5, 12),
        "random_state": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    }
    clf = GridSearchCV(MLPClassifier(), parameters, n_jobs=-1)
    clf.fit(train_sample, train_label)
    test_predict = clf.predict(test_sample)
    # 打印最佳参数
    print("Best parameters found: ", clf.best_params_)
    return test_predict

In [9]:
# 测试iris_predict
iris = datasets.load_iris()
iris_data = iris.data
iris_label = iris.target
test_predict = iris_predict(iris_data, iris_label, iris_data)
print(test_predict)
# 计算准确率
correct = np.sum(test_predict == iris_label)
n = len(test_predict)
accuracy = correct / n
print("准确率为：%.2f%%" % (accuracy * 100))

Best parameters found:  {'alpha': 0.001, 'hidden_layer_sizes': 6, 'max_iter': 500, 'random_state': 0, 'solver': 'lbfgs'}
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
准确率为：98.67%
